# Úlohy

## Výpočet ceny jízdenky

### Vstupy:
- `vek` – věk cestujícího (např. 30)
- `student` – je student? (`True` nebo `False`)
- `duchodce` – je důchodce? (`True` nebo `False`)
- `ztp` – je držitel průkazu ZTP? (`True` nebo `False`)
- `vzdalenost` – vzdálenost v kilometrech (např. 120)
- `typ_spoje` – typ spoje (`"osobni"`, `"rychlik"`, `"expres"`)

---

### Pravidla:

#### 1. Základní cena jízdenky:
Podle typu spoje:
- `osobni` → 1.50 Kč/km  
- `rychlik` → 2.00 Kč/km  
- `expres` → 2.50 Kč/km  

#### 2. Slevy:
Sleva se počítá ze základní ceny:

| Podmínka                          | Sleva     |
|----------------------------------|-----------|
| Věk < 6                          | 100 %     |
| Držitel ZTP                      | 100 %     |
| Věk < 15                         | 50 %      |
| Student do 26 let                | 25 %      |
| Důchodce nebo věk ≥ 65           | 50 %      |

#### 3. Výběr slevy:
- Pokud platí více slev, použije se ta **největší**.
- Sleva se aplikuje pouze jednou.

---

### Příklady:

- **Pokud** `vek < 6` → cena = 0 Kč  
- **Jinak pokud** `ztp == True` → cena = 0 Kč  
- **Jinak pokud** `vek < 15` → sleva = 50 %  
- **Jinak pokud** `student == True` a `vek <= 26` → sleva = 25 %  
- **Jinak pokud** `duchodce == True` nebo `vek >= 65` → sleva = 50 %  
- **Jinak** → žádná sleva, platí plná cena.

In [ ]:
def cena_jizdenky(vek, student, duchodce, ztp, vzdalenost, typ_spoje):
    # sazby podle typu spoje
    sazby = {
        "osobni": 1.5,
        "rychlik": 2.0,
        "expres": 2.5
    }

    # základní cena
    sazba = sazby.get(typ_spoje, 2.0)  # výchozí rychlík
    zakladni_cena = vzdalenost * sazba

    # pravidla pro slevy
    if vek < 6 or ztp:
        return 0.0

    slevy = []

    if vek < 15:
        slevy.append(0.5)
    if student and vek <= 26:
        slevy.append(0.25)
    if duchodce or vek >= 65:
        slevy.append(0.5)

    if slevy:
        max_sleva = max(slevy)
        return round(zakladni_cena * (1 - max_sleva), 2)
    else:
        return round(zakladni_cena, 2)

# ---------- Interaktivní dotazování ----------

def ano_ne(prompt):
    """Pomocná funkce pro zadání ano/ne"""
    while True:
        odpoved = input(prompt + " (ano/ne): ").strip().lower()
        if odpoved in ["ano", "a"]:
            return True
        elif odpoved in ["ne", "n"]:
            return False
        else:
            print("Zadej prosím 'ano' nebo 'ne'.")

def hlavni_dotazovani():
    print("Vítej v kalkulačce ceny jízdenky! ")

    try:
        vek = int(input("Zadej svůj věk: "))
        student = ano_ne("Jsi student?")
        duchodce = ano_ne("Jsi důchodce?")
        ztp = ano_ne("Máš průkaz ZTP?")
        vzdalenost = float(input("Zadej vzdálenost v km: "))

        while True:
            typ_spoje = input("Zadej typ spoje (osobni, rychlik, expres): ").strip().lower()
            if typ_spoje in ["osobni", "rychlik", "expres"]:
                break
            else:
                print("Neplatný typ spoje. Zadej 'osobni', 'rychlik' nebo 'expres'.")

        cena = cena_jizdenky(vek, student, duchodce, ztp, vzdalenost, typ_spoje)
        print(f"\n Cena jízdenky je {cena:.2f} Kč")

    except ValueError:
        print("Chyba vstupu: zkontroluj, že věk a vzdálenost jsou čísla.")

# Spuštění dotazování
hlavni_dotazovani()

Vítej v kalkulačce ceny jízdenky! 
Zadej svůj věk: 36
Jsi student? (ano/ne): ne
Jsi důchodce? (ano/ne): ne
Máš průkaz ZTP? (ano/ne): ne
Zadej vzdálenost v km: 96
Zadej typ spoje (osobni, rychlik, expres): osobni

 Cena jízdenky je 144.00 Kč


## Chytrý protivník na kámen-nůžky-papír


V tomto úkolu naprogramuješ **učícího se protivníka**, který bude hrát hru **Kámen–Nůžky–Papír** proti tobě. Na rozdíl od náhodného chování se tvůj algoritmus bude **učit podle toho, co obvykle hraješ**.

---

## Cíl programu:
Vytvoř program, který:
- umožní hráči zadávat tahy (`kamen`, `nuzky`, `papir`)
- rozhodne, kdo vyhrál (hráč nebo počítač)
- **učí se z historie tahů hráče** a podle toho volí tahy tak, aby ho co nejčastěji porazil
- opakuje hru, dokud hráč nenapíše `"konec"`

---

## Požadavky na program:

1. **Zpracuj vstup** od uživatele:
   - hráč může zadat `kamen`, `nuzky`, `papir`, nebo `konec`
   - pokud zadá neplatný vstup, vypiš upozornění

2. **Vytvoď jednoduché rozhodovací pravidlo**, které vybere tah počítače podle **dosavadní historie hráče**:
   - například: pokud hráč nejčastěji hraje `kamen`, počítač by měl zvolit `papir`, protože ten ho porazí

3. **Zobraz výsledek kola**:
   - vypiš tah hráče a počítače
   - napiš, kdo vyhrál (`Vyhrál hráč`, `Vyhrál počítač`, `Remíza`)

4. **Umožni více herních kol**, dokud hráč nezadá `"konec"`

---

## Tipy:

- Můžeš si pamatovat historii tahů hráče pomocí slovníku:

  `historie = {"kamen": 0, "nuzky": 0, "papir": 0}`

In [ ]:
import random

# Pravidla hry: co poráží co
vyhra_nad = {
    "kamen": "nuzky",
    "nuzky": "papir",
    "papir": "kamen"
}

# Historie tahů hráče
historie = {
    "kamen": 0,
    "nuzky": 0,
    "papir": 0
}

def tah_pocitace():
    # Pokud je historie prázdná, hraj náhodně
    if sum(historie.values()) == 0:
        return random.choice(["kamen", "nuzky", "papir"])

    # Najdi nejčastější volbu hráče
    nejcastejsi_hracuv_tah = max(historie, key=historie.get)

    # Vyber tah, který ho porazí
    for tah, porazeny in vyhra_nad.items():
        if porazeny == nejcastejsi_hracuv_tah:
            return tah

def vyhodnot(hrac, pocitac):
    if hrac == pocitac:
        return "Remíza"
    elif vyhra_nad[hrac] == pocitac:
        return "Vyhrál hráč!"
    else:
        return "Vyhrál počítač!"

# Hlavní smyčka hry
while True:
    hracuv_tah = input("Zadej svůj tah (kamen/nuzky/papir nebo konec): ").lower()
    if hracuv_tah == "konec":
        break
    if hracuv_tah not in ["kamen", "nuzky", "papir"]:
        print("Neplatný tah. Zkus to znovu.")
        continue

    # Zaznamenej tah hráče
    historie[hracuv_tah] += 1

    # Počítač hraje na základě historie
    pocitacuv_tah = tah_pocitace()

    # Výstup
    print(f"Hráč: {hracuv_tah} | Počítač: {pocitacuv_tah}")
    print(vyhodnot(hracuv_tah, pocitacuv_tah))
    print("-" * 30)

Hráč: kamen | Počítač: papir
Vyhrál počítač!
------------------------------
Hráč: kamen | Počítač: papir
Vyhrál počítač!
------------------------------
Hráč: nuzky | Počítač: papir
Vyhrál hráč!
------------------------------
Hráč: nuzky | Počítač: papir
Vyhrál hráč!
------------------------------
Hráč: nuzky | Počítač: kamen
Vyhrál počítač!
------------------------------
Hráč: nuzky | Počítač: kamen
Vyhrál počítač!
------------------------------
Zadej svůj tah (kamen/nuzky/papir nebo konec): konec


# Obchodní cestující

Představ si, že máš seznam měst, která musíš navštívit a vrátit se zpět do města kde jsi začal.

1. Vyzkoušej náhodné pořadí návštěv a zapamatuj si to nejlepší, co jsi zatím našel. Opakuj to třeba 100×.
2. Prohledej všechny možné trasy
3. K zamyšlení - šlo by to udělat chytřeji?

In [ ]:
import random
import math

mesta = {
    "A": (1, 0),
    "B": (2, 3),
    "C": (5, 2),
    "D": (6, 8),
    "E": (8, 3),
    "F": (2, 1),
    "G": (4, 9),
    "H": (0, 9),
    "I": (1, 2),
}

def vzdalenost(a, b):
    x1, y1 = mesta[a]
    x2, y2 = mesta[b]
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def delka_trasy(trasa):
    trasa = trasa + [trasa[0]]  # návrat domů
    return sum(vzdalenost(trasa[i], trasa[i+1]) for i in range(len(trasa)-1))

nejlepsi_trasa = None
nejlepsi_delka = float('inf')

for _ in range(1000):  # můžeš zkusit i méně
    trasa = list(mesta.keys())
    random.shuffle(trasa)
    d = delka_trasy(trasa)
    if d < nejlepsi_delka:
        nejlepsi_trasa = trasa
        nejlepsi_delka = d

print("Nejlepší trasa:", nejlepsi_trasa)
print("Délka:", round(nejlepsi_delka, 2))

Nejlepší trasa: ['C', 'A', 'F', 'I', 'B', 'D', 'G', 'H', 'E']
Délka: 34.52
